In [51]:
#set up our imports 
import pandas as pd

def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson
#load some data into a dataframe
wd = [31, 37, 9, 9, 9, 9, 6, 6, 104]
cols = ['point', 'prov', 'latg', 'latm', 'lngg', 'lngm', 'h', 'number', 'oaci']

dfp = pd.read_fwf('estaciones_smn.txt', widths=wd, names=cols)

dfp = dfp[2:]
dt = {
    'point': str, 'prov': str, 
    'latg': int, 'latm': int, 'lngg': int, 'lngm': int, 
    'h': int, 'number': int,
    'oaci': str
}

for k in dt:
    dfp[k] = dfp[k].astype(dt[k])

dfp['lat'] = dfp['latg'] - (dfp['latm'] / 60)
dfp['lng'] = dfp['lngg'] - (dfp['lngm'] / 60)


v = [8, 6, 6, 101-8-12]
dft = pd.read_fwf("temps.txt", widths=v, names=['date', 'max', 'min', 'loc'])
dft = dft[3:]

dft["ansidate"] = dft.loc[:, "date"].apply(lambda x: x[4:]+x[2:4]+x[:2])
del(dft['date'])
dft['max'] = dft['max'].astype('float')
dft['min'] = dft['min'].astype('float')
dft = dft.iloc[::-1]

geojson_markers = df_to_geojson(dfp[['point', 'lat', 'lng', 'h']], {'point', 'h'}, lat='lat', lon='lng')
#let's look at the dataframe head
dfp


,point,prov,latg,latm,lngg,lngm,h,number,oaci,lat,lng
2,BASE BELGRANO II,ANTARTIDA,-77,52,-34,34,256,89034,SAYB,-77.866667,-34.566667
3,BASE CARLINI (EX JUBANY),ANTARTIDA,-62,14,-58,38,11,89053,SAYJ,-62.233333,-58.633333
4,BASE ESPERANZA,ANTARTIDA,-63,24,-56,59,24,88963,SAYE,-63.400000,-56.983333
5,BASE MARAMBIO,ANTARTIDA,-64,14,-56,43,198,89055,SAWB,-64.233333,-56.716667
6,BASE ORCADAS,ANTARTIDA,-60,45,-44,43,12,88968,SAYO,-60.750000,-44.716667
7,BASE SAN MARTIN,ANTARTIDA,-68,8,-67,8,7,89066,SAYS,-68.133333,-67.133333
8,AZUL AERO,BUENOS AIRES,-36,50,-59,53,147,87641,SAZA,-36.833333,-59.883333
9,BAHIA BLANCA AERO,BUENOS AIRES,-38,44,-62,10,83,87750,SAZB,-38.733333,-62.166667
10,BENITO JUAREZ AERO,BUENOS AIRES,-37,43,-59,47,207,87649,SAZJ,-37.716667,-59.783333
11,BOLIVAR AERO,BUENOS AIRES,-36,12,-61,4,94,87640,SAZI,-36.200000,-61.066667


In [52]:
%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from ipywidgets import Layout, HBox, VBox, widgets
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    LayersControl, Marker, MarkerCluster,
    GeoJSON, FullScreenControl, WidgetControl
)


defaultLayout=Layout(width='100%', height='1024px')
out1 = widgets.Output()
from_date = widgets.DatePicker(description='From', disabled=False)
to_date = widgets.DatePicker(description='To', disabled=False)

def clear_plt(*args, **kwargs):
    plt.cla()
    plt.clf()
    plt.close()
    out1.clear_output()


def mark_click(*args, **kwargs):
    global out1
    print(args, kwargs)
    if kwargs.get("event", "other") == "click":
        props = kwargs.get("properties", {})
        point = props.get("point", None)
        if point is not None:
            data = dft[dft["loc"].str.startswith(point)]
            if from_date.value is not None:
                frm_value = int(''.join(from_date.value.isoformat().split('-')))
                data.drop(data[data['ansidate'].astype('int64') < frm_value].index, inplace=True)
            if to_date.value is not None:
                to_value = int(''.join(to_date.value.isoformat().split('-')))
                data.drop(data[data['ansidate'].astype('int64') > to_value].index, inplace=True)
            data.reset_index(inplace=True)
            del data['index']
            minim = min(data['min'])
            maxam = max(data['max'])
            ranged = np.arange(minim, maxam+1, 1.5)
            ax = data.plot(x='ansidate', y=['min', 'max'])
            ax.set(xlabel="Date", ylabel="Temp")
            plt.yticks(ranged)
            plt.xticks(rotation=90)
            with out1:
                plt.title(f"{point} - Alt.: {props.get('h', 'No Data')}")
                plt.show()


geo_json = GeoJSON(data=geojson_markers,
                   name="Datos del SMN",
                   style={'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2})
geo_json.on_click(mark_click)

btn = widgets.Button(description="Clear")
btn.on_click(clear_plt)

m = Map(center=(dfp.lat.mean(), dfp.lng.mean()), zoom=6, layout=defaultLayout)
today = datetime.today().date()
nasa_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, (today - timedelta(days=1)).isoformat())
    
m.add_layer(nasa_layer)
m.add_layer(geo_json)

m.add_control(WidgetControl(widget=to_date, position="bottomleft"))
m.add_control(WidgetControl(widget=from_date, position="bottomleft"))
m.add_control(WidgetControl(widget=btn, position='bottomright'))
m.add_control(WidgetControl(widget=out1, position='bottomright'))
m.add_control(FullScreenControl())
m.add_control(LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

() {'event': 'click', 'properties': {'point': 'VIEDMA AERO', 'h': 7}}
() {'event': 'interaction', 'type': 'click', 'coordinates': [-40.85, -63.016666666666666]}
() {'event': 'click', 'properties': {'point': 'SAN ANTONIO OESTE AERO', 'h': 20}}
() {'event': 'interaction', 'type': 'click', 'coordinates': [-40.78333333333333, -65.1]}
